In [1]:
from pathlib import Path
from math import ceil

import pandas as pd
import torch
import pytorch_lightning as pl

from classification.train_base import MultiPartitioningClassifier # class defining our model
from classification.dataset import FiveCropImageDataset # class for preparing the images before giving them to the NN

## Load the model

In [2]:
# where model's params and hyperparams are saved
checkpoint = "models/base_M/epoch=014-val_loss=18.4833.ckpt"
hparams = "models/base_M/hparams.yaml"

In [3]:
# load_from_checkpoint is a static method from pytorch lightning, inherited by MultiPartitioningClassifier
# it permits to load a model previously saved, in the form of a checkpoint file, and one with hyperparameters
# MultiPartitioningClassifier is the class defining our model
model = MultiPartitioningClassifier.load_from_checkpoint(
    checkpoint_path=checkpoint,
    hparams_file=hparams,
    map_location=None
)

#model.eval() to see the model's structure

In [4]:
#to allow GPU
want_gpu = True
if want_gpu and torch.cuda.is_available():
    gpu = 1
else:
    gpu = None

# the class Trainer from pythorch lightining is the one responsible for training a deep NN
# it can initialize the model, run forward and backward passes, optimize, print stats, early stop...
wanted_precision = 32 #16 for half precision (how many bits for each number)
trainer = pl.Trainer(gpus=gpu, precision=wanted_precision)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores


## Load and initialize the images

In [5]:
# where images are saved
image_dir = "resources/images/im2gps"
meta_csv = "resources/images/im2gps_places365.csv"

In [6]:
#FiveCropImageDataset is the class for preparing the images before giving them to the NN
# in particular, it creates five different crops for every image
dataset = FiveCropImageDataset(meta_csv, image_dir)

Read resources/images/im2gps_places365.csv


In [7]:
batch_size = 64
dataloader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=ceil(batch_size / 5),  #you divide by 5 because for each image you generate 5 different crops
                    shuffle=False,
                    num_workers=4 #number ot threads used for parallelism (cores of CPU?)
                )

## Run the model on the test set

In [8]:
results = trainer.test(model, test_dataloaders=dataloader, verbose=False)

Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [02:49<00:00,  8.93s/it]


/home/db/anaconda3/envs/geoestimation-github-pytorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The testing_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


## Look at the results

In [20]:
# formatting results into a pandas dataframe
df = pd.DataFrame(results[0]).T
#df["dataset"] = image_dir
df["partitioning"] = df.index
df["partitioning"] = df["partitioning"].apply(lambda x: x.split("/")[-1])
df.set_index(keys=["partitioning"], inplace=True) #keys=["dataset", "partitioning"] in case
print(df)

                  1         25        200       750       2500
partitioning                                                  
coarse        0.092827  0.316456  0.497890  0.670886  0.789030
middle        0.139241  0.345992  0.481013  0.683544  0.793249
fine          0.156118  0.392405  0.489451  0.658228  0.784810
hierarchy     0.147679  0.375527  0.489451  0.683544  0.789030


In [21]:
# to save the dataframe on a csv file
fout = 'test_results.csv'
df.to_csv(fout)